In [1]:
import os
import json
import re
import xml.etree.ElementTree as ET
from typing import Dict, List


In [2]:
lines = []
for file in os.listdir('Sohas_weapon-Detection/annotations/xmls'):
    lines.append('Sohas_weapon-Detection/annotations/xmls/' + file)
with open('knives.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
f.close()

In [3]:
with open('labels.txt', 'w') as f:
    f.write('knife')
f.close()



In [4]:
def get_label2id(labels_path: str) -> Dict[str, int]:
    with open(labels_path, 'r') as f:
        labels_str = f.read().split()
    labels_ids = list(range(1, len(labels_str)+1))
    return dict(zip(labels_str, labels_ids))


def get_annpaths(ann_dir_path: str = None,
                 ann_ids_path: str = None,
                 ext: str = '',
                 annpaths_list_path: str = None) -> List[str]:
    # If use annotation paths list
    if annpaths_list_path is not None:
        with open(annpaths_list_path, 'r') as f:
            ann_paths = f.read().split()
        return ann_paths

    # If use annotaion ids list
    ext_with_dot = '.' + ext if ext != '' else ''
    with open(ann_ids_path, 'r') as f:
        ann_ids = f.read().split()
    ann_paths = [os.path.join(ann_dir_path, aid+ext_with_dot) for aid in ann_ids]
    return ann_paths


def get_image_info(annotation_root, extract_num_from_imgid=True):
    path = annotation_root.findtext('path')
    if path is None:
        filename = annotation_root.findtext('filename')
    else:
        filename = os.path.basename(path)
    for counter, char in enumerate(reversed(filename)):
        if char == '\\':
            filename = filename[-counter:]
            break
            
    img_name = os.path.basename(filename)
    img_id = os.path.splitext(img_name)[0]
    if extract_num_from_imgid and isinstance(img_id, str):
        img_id = int(re.findall(r'\d+', img_id)[0])

    size = annotation_root.find('size')
    width = int(size.findtext('width'))
    height = int(size.findtext('height'))

    image_info = {
        'id': str(filename).replace('.jpg',''),
        'width': width,
        'height': height,
        'file_name': filename,
    }
    if filename.find('billete') == -1:
        return image_info
    else:
        return ''

def get_coco_annotation_from_obj(obj, label2id):
    label = obj.findtext('name')
#     assert label in label2id, f"Error: {label} is not in label2id !"
    if label == 'knife':
        category_id = label2id[label]
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.findtext('xmin'))) - 1
        ymin = int(float(bndbox.findtext('ymin'))) - 1
        xmax = int(float(bndbox.findtext('xmax')))
        ymax = int(float(bndbox.findtext('ymax')))
        assert xmax > xmin and ymax > ymin, f"Box size error !: (xmin, ymin, xmax, ymax): {xmin, ymin, xmax, ymax}"
        o_width = xmax - xmin
        o_height = ymax - ymin
        ann = {
            'category_id': category_id,
            'segmentation': [],  # This script is not for segmentation
            'area': o_width * o_height,
            'bbox': [xmin, ymin, o_width, o_height],
            'iscrowd': 0,
        }
        return ann


def convert_xmls_to_cocojson(annotation_paths: List[str],\
                             label2id: Dict[str, int],
                             output_jsonpath: str,
                             extract_num_from_imgid: bool = True):
    output_json_dict = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    bnd_id = 1  # START_BOUNDING_BOX_ID, TODO input as args ?
    print('Start converting !')
    for a_path in annotation_paths:
        # Read annotation xml
        ann_tree = ET.parse(a_path)
        ann_root = ann_tree.getroot()

        img_info = get_image_info(annotation_root=ann_root,
                                  extract_num_from_imgid=extract_num_from_imgid)
        if img_info:
            img_id = img_info['id']
            output_json_dict['images'].append(img_info)

        for obj in ann_root.findall('object'):
            ann = get_coco_annotation_from_obj(obj=obj, label2id=label2id)
            if ann:
                annot = {'id': bnd_id, 'image_id': img_id,}
                annot.update(ann)
                output_json_dict['annotations'].append(annot)
                bnd_id = bnd_id + 1

    for label, label_id in label2id.items():
        category_info = {'id': label_id, 'name': label, 'supercategory': 'none'}
        output_json_dict['categories'].append(category_info)

    with open(output_jsonpath, 'w') as f:
        output_json = json.dumps(output_json_dict)
        f.write(output_json)
        
def convert_to_coco(ann_path_list='./knives.txt',\
                    labels='./labels.txt', \
                    output='./output1.json'):
    label2id = get_label2id(labels_path=labels)
    ann_paths = get_annpaths(
        annpaths_list_path=ann_path_list
    )
    convert_xmls_to_cocojson(
        annotation_paths=ann_paths,
        label2id=label2id,
        output_jsonpath=output
    )

In [5]:
convert_to_coco()

Start converting !


In [6]:
import torch, detectron2

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
print(TORCH_VERSION, detectron2.__version__)

1.10 0.6


In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

#some common libs
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# some utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
##from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

ImportError: /home/lhue/anaconda3/lib/python3.9/site-packages/detectron2/_C.cpython-39-x86_64-linux-gnu.so: undefined symbol: _ZNK2at6Tensor7is_cudaEv

In [6]:
import torch, detectron2

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
print(TORCH_VERSION, detectron2.__version__)

1.10 0.6


In [7]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

#some common libs
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# some utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

ImportError: /home/lhue/anaconda3/lib/python3.9/site-packages/detectron2/_C.cpython-39-x86_64-linux-gnu.so: undefined symbol: _ZNK2at6Tensor7is_cudaEv

In [8]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("k", {}, "./output1.json", "../input/weapon-detection-dataset/Sohas_weapon-Detection/images/")

ImportError: /home/lhue/anaconda3/lib/python3.9/site-packages/detectron2/_C.cpython-39-x86_64-linux-gnu.so: undefined symbol: _ZNK2at6Tensor7is_cudaEv

In [9]:
knives_metadata = MetadataCatalog.get('k')
dataset_dict_knives = DatasetCatalog.get('k')

NameError: name 'MetadataCatalog' is not defined

In [ ]:
import matplotlib.pyplot as plt
for d in random.sample(dataset_dict_knives, 10):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata = knives_metadata, scale=2)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("k",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.0001  # pick a good LR
cfg.SOLVER.MAX_ITER = 2000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32   # using 64 as batch size
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # we are currently dealing with only grenade class

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
# predictor = DefaultPredictor(cfg)

In [ ]:
f = open('config_knives.yml', 'w')
f.write(cfg.dump())
f.close()

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("config_knives.yml") #configuration file
cfg.MODEL.DEVICE='cpu'
cfg.MODEL.WEIGHTS = ("output/model_final.pth") #path of the trained model
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.7 # For RETINANET Model
predictor1 = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
im1 = plt.imread('../input/weapon-detection-dataset/Sohas_weapon-Detection/images_test/knife_1186.jpg')
# im1 = cv2.imread('/content/g2.jpg')
#im1 = cv2.imread('/content/test5.jpg')

outputs1 = predictor1(im1)
v = Visualizer(im1[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
threshed=[]
for i in range(0, len(outputs1['instances'].scores)):
    if outputs1['instances'].scores[i]>0.75:
        threshed.append(i)

v = v.draw_instance_predictions(outputs1["instances"][threshed].to("cpu"))
plt.figure(figsize=(20,20))
plt.imshow(v.get_image()[:, :, ::-1])